<a href="https://colab.research.google.com/github/farheenfab/DMML-GroupProject/blob/Ishaq/Lab/Lab%204/dataset_explore_lab_4_group_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import requests
import zipfile
from sklearn.model_selection import train_test_split

dataset_url = 'https://github.com/ishaqmarashy/DATALFS/raw/main/JMuBEN.zip'
dataset_dir = './JMuBEN'

# create directory for dataset if it does not exist
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)
# append JMuBEN.zip to the end of the path (this is where we download the file to)
zip_file_path = os.path.join(dataset_dir, 'JMuBEN.zip')

# check if file is downloaded already
if not os.path.exists(zip_file_path):
    # file is not downloaded so fetch the file
    response = requests.get(dataset_url)
    # write file to storage which is recieved from the response
    with open(zip_file_path, 'wb') as zip_file:
        zip_file.write(response.content)

# unzip to zip file path
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(dataset_dir)
# within the concat train and test to become ./JMuBEN/train and JMuBEN ./JMuBEN/test
train_dir = os.path.join(dataset_dir, 'train')
test_dir = os.path.join(dataset_dir, 'test')

In [5]:
def load_images_and_labels(directory):
    images = []
    labels = []
    # get subdirectories Healthy and Miner
    for class_name in os.listdir(directory):
        # concat subdirectory to get full path
        class_dir = os.path.join(directory, class_name)
        # assign labels using class subdirectory
        # label is determined by filepath
        label = 0 if class_name == 'Miner' else 1
        # append labels and image paths to labels and images respectively
        for filename in os.listdir(class_dir):
            images.append(os.path.join(class_dir, filename))
            labels.append(label)
    return images, labels

# load file directories and their labels
train_images, train_labels = load_images_and_labels(train_dir)
test_images, test_labels = load_images_and_labels(test_dir)

In [6]:
# unnecessary but we can farther split the training data using scikitlearn to smaller test train sets
X_train, X_test, y_train, y_test = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# print the number of images and labels
print(f"Train :{len(X_train)}")
print(f"Test :{len(X_test)}")

Train :19200
Test :4800
